<a href="https://colab.research.google.com/github/nabin-nath/major-project-2/blob/electricity-consumption/RandomSearchTune_e_theft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install -U keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.1 MB/s eta 0:00:00


In [ ]:
#imports
import pandas as pd
import numpy as np

df0 = pd.read_csv('/content/drive/MyDrive/Major Project - II/0_interpolate_n.csv')
df1 = pd.read_csv('/content/drive/MyDrive/Major Project - II/1_interpolate_n.csv')

/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252

In [ ]:
df0 = df0.iloc[:,2:]
df1 = df1.iloc[:3579,2:]
df0 = df0.drop(df0.index[0])
df1 = df1.drop(df1.index[0])

df0 = df0.fillna(0)
df0 = df0.astype('float64')
df1 = df1.fillna(0)
df1 = df1.astype('float64')

from sklearn.utils import resample
df1 = resample(df1,
             replace=True,
             n_samples=len(df0),
             random_state=42)
             
df = pd.concat([df0, df1], ignore_index=True)

In [ ]:
# dataset shuffling
df = df.sample(frac=1).reset_index(drop=True)

# test and train dataset preparation
trainX, trainy = df.iloc[:60000, 1:], df.iloc[:60000, :1]
testX, testy = df.iloc[60001:, 1:], df.iloc[60001:, :1]

# print(trainX.shape, trainy.shape, testX.shape, testy.shape)

In [ ]:
#input scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(trainX)
scaled_train = scaler.transform(trainX)
scaled_test = scaler.transform(testX)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from tensorflow import keras
# from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras_tuner.tuners import RandomSearch

In [ ]:
def build_model(hp):
  n_timesteps, n_features, n_outputs = trainX.shape[1], 1, trainy.shape[1]
  model = Sequential()
  model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
  model.add(Dropout(hp.Choice('dropout', [0.2, 0.5])))

  for i in range(hp.Int('layers', 1, 4)):
    model.add(Dense(
      units=hp.Int('units_' + str(i), 30, 500, step=30),
      activation=hp.Choice('act_' + str(i), ['relu', 'sigmoid'])    
    ))

  model.add(Dense(n_outputs, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(
      hp.Choice('learning_rate',
                values=[1e-2, 1e-4])),
                metrics=['accuracy'])

  return model

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=4,
    executions_per_trial=3,
    directory='my_dir',
    project_name='rnn_lstm'
)

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 5
dropout (Choice)
{'default': 0.2, 'conditions': [], 'values': [0.2, 0.5], 'ordered': True}
layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 4, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 30, 'max_value': 500, 'step': 30, 'sampling': 'linear'}
act_0 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.0001], 'ordered': True}


In [ ]:
tuner.search(trainX, trainy,
             epochs=15,
             validation_data=(testX,testy))

Trial 3 Complete [00h 55m 17s]
val_accuracy: 0.6673404773076376

Best val_accuracy So Far: 0.6673404773076376
Total elapsed time: 02h 43m 14s

Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
0.5               |0.2               |dropout
3                 |1                 |layers
210               |120               |units_0
sigmoid           |sigmoid           |act_0
0.0001            |0.0001            |learning_rate
390               |450               |units_1
sigmoid           |relu              |act_1
60                |30                |units_2
relu              |sigmoid           |act_2
300               |450               |units_3
sigmoid           |sigmoid           |act_3

Epoch 1/15
1875/1875 [==============================] - 78s 40ms/step - loss: 0.6513 - accuracy: 0.6162 - val_loss: 0.6405 - val_accuracy: 0.6388
Epoch 2/15
1875/1875 [==============================] - 69s 37ms/step - loss: 0.6386 - accuracy: 0.6377 - val_loss: 0.6331 - val

In [ ]:
tuner.results_summary()